**예제 1. Generator 구성하기**

**예제 2. Discriminator 구성하기**

**예제 3. 구성한 Generator 와 Discriminator 학습해 새로운 MNIST 이미지 생성**


*   필요한 Library Import




In [ ]:
import os
import numpy as np
import utils
import time
import torch
import torch.nn as nn
import torch.autograd as autograd

from IPython.display import Image as Display
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
from PIL import Image
import matplotlib.pyplot as plt

*   학습에 사용될 Hyper Parameter 설정

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("Using Device:", DEVICE)

epochs = 200
batch_size = 100
latent_dim = 100
lr = 0.0002
save_term = 500 
start_time = time.time()

*   학습에 필요한 MNIST 데이터셋 다운로드

In [ ]:
transform = transforms.Compose([
        transforms.Resize([64, 64]), 
        transforms.Grayscale(1),
        transforms.ToTensor(),                     
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

cifar10_dataset = datasets.CIFAR10(root='./data/', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(cifar10_dataset, batch_size= batch_size, shuffle=True)

*   Generator 구성

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.embed = nn.Embedding(10, 10)

        def block(input, output, normalize=True):
            layer= [nn.Linear(input, output)]
            if normalize:
                layer.append(nn.BatchNorm1d(output, 0.8))
            layer.append(nn.LeakyReLU(0.2, inplace=True))
            return layer

        self.model = nn.Sequential(
            *block(110, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 1 * 64 * 64),
            nn.Tanh()
        )

    def forward(self, z, label):
        c = self.embed(label)
        x = torch.cat((z, c), -1)
        img = self.model(x)
        img = img.view(img.size(0), 1, 64, 64)
        return img

*   Discriminator 구성

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.embed = nn.Embedding(10, 10)

        self.model = nn.Sequential(
            nn.Linear(1 * 64 * 64 + 10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, x, label):
        x2 = x.view(x.size(0), -1)
        c = self.embed(label)
        inputs = torch.cat((x2, c), -1)
        out = self.model(inputs)
        return out

*   Loss Function 과 Optimization 정의

In [ ]:
generator = Generator()
discriminator = Discriminator()
generator.cuda()
discriminator.cuda()

criterion = nn.BCELoss()
criterion.cuda()

g_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

In [ ]:
Tensor = torch.cuda.FloatTensor
LongTensor = torch.cuda.LongTensor

def gradient_compute(Discriminator, real, fake, label):
    alpha = Tensor(np.random.random((real.size(0), 1, 1, 1)))
    label = LongTensor(label)
    grad_input = (alpha * real + ((1 - alpha) * fake)).requires_grad_(True)
    grad_out = Discriminator(grad_input, label)
    fake = Tensor(real.shape[0], 1).fill_(1.0)
    fake.requires_grad = False

    gradients = autograd.grad(
        outputs=grad_input,
        inputs=grad_input,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )

    gradients = gradients[0].view(gradients[0].size(0), -1)
    results= ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return results


*   학습

In [ ]:
for epoch in range(epochs):
    for i, (images, label) in enumerate(train_dataloader):
        real_image = images.cuda()
        label = label.cuda()
        optimizer_D.zero_grad()
        z = torch.normal(mean=0, std=1, size=(images.shape[0], latent_dim)).cuda()
        fake_image = generator(z, label)
        
        real_out = discriminator(real_image, label)
        # Fake images
        fake_out = discriminator(fake_image, label)
        # Gradient penalty
        gradient = gradient_compute(
                            discriminator, real_image.data, fake_image.data,
                            label.data)
        
        loss_D = -torch.mean(real_out) + torch.mean(fake_out) + 10 * gradient
        loss_D.backward()
        optimizer_D.step()
        
        optimizer_G.zero_grad()


        if i % 5 == 0:
            fake_image2 = wg_generator(z, label)
            fake_out2 = wg_discriminator(fake_image2, label)
            loss_G = -torch.mean(fake_out2)
            loss_G.backward()
            optimizer_G.step()
        
    print(f"[Epoch {epoch}/{epochs}] [loss_D: {loss_D.item():.6f}] [loss_G: {loss_G.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")

*   생성하고 싶은 숫자 정한 후 시각화 및 학습결과 확인

In [ ]:
z = torch.normal(mean=0, std=1, size=(100, latent_dim)).cuda()
g_label = torch.cuda.IntTensor(100).fill_(0)
fake_image = generator(z, g_label)

fake_out = np.reshape(fake_image.data.cpu().numpy()
                               [0],(64, 64))

plt.imshow(fake_out, cmap = 'gray')
plt.show()